<a href="https://colab.research.google.com/github/anvargasch/IEStoPBRT/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the repository containing the conversion functions and sample IES files

In [ ]:
!git clone https://github.com/anvargasch/IEStoPBRT.git
%cd IEStoPBRT

# Install dependencies

In [ ]:
!pip -q install numpy imageio opencv-python

# Patch EXR saving in Colab (fallback to OpenCV if imageio fails)

In [ ]:
import numpy as np
import cv2
import imageio.v3 as iio

# Save original imwrite
_iio_orig_imwrite = iio.imwrite

def _imwrite_exr_patch(uri, img, **kwargs):
    if str(uri).lower().endswith(".exr"):
        ok = cv2.imwrite(str(uri), img.astype(np.float32))
        if not ok:
            raise RuntimeError(f"OpenCV couldn't write EXR: {uri}")
    else:
        _iio_orig_imwrite(uri, img, **kwargs)

# Apply patch
iio.imwrite = _imwrite_exr_patch
print("✅ Patched EXR saving (using OpenCV fallback)")

# Import the converter

In [ ]:
import sys
sys.path.append("/content/IEStoPBRT")

import ies2exr

# Example: convert an IES file into an EXR

In [ ]:
filename = "IESfiles/IESsiemens.ies"   # Replace with your own IES file
output   = "IESsiemens.exr"

meta = ies2exr.ies_to_exr(filename, output, width=1024, height=512)
print("Metadata:", meta)

# Visualize the generated photometric image

In [ ]:
import imageio.v3 as iio
import matplotlib.pyplot as plt

img = iio.imread(output)
plt.figure(figsize=(12,4))
plt.imshow(img[:,:,0], cmap='gray')
plt.title('Goniometric Image for PBRT')
plt.colorbar(label='Candela values')
plt.show()

# The EXR file is now ready to be used in PBRT as a goniometric light source.